# 400 - gradient et backward

On peut avoir un réseau de neurones comme un gros sandwitch avec de multiples couches de neurones dans lequel on a envie d'insérer sa propre couche mais pour cela il faut interférer avec le gradient. J'ai repris le tutoriel [pytorch: defining new autograd functions](https://pytorch.org/tutorials/beginner/examples_autograd/two_layer_net_custom_function.html). L'exemple suivant [Extending Torch](https://pytorch.org/docs/master/notes/extending.html). J'aimais bien l'API de la version 0.4 mais je ne la trouve plus sur internet. Elle laissait sans doute un peu trop de liberté.

In [1]:
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
print("torch", torch.__version__)
from torchvision import datasets, transforms
from tqdm import tqdm

torch 1.0.1


In [2]:
%matplotlib inline

In [3]:
BATCH_SIZE = 64
TEST_BATCH_SIZE = 64
DATA_DIR = 'data/'
USE_CUDA = False  # switch to True if you have GPU
N_EPOCHS = 2 # 5

In [4]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Couche personnalisée

L'exemple suivant illustre comment définir une couche intermédiaire qui obéit à ses propres règles. Elle doit implémenter les deux méthodes ``forward`` qui calcule la prédiction pour la couche suivante et ``backward`` pour le calcul du gradient pour la couche précédente. Il reste la variable ``ctx`` qui permet de stocker des informations qui persistent jusqu'au calcul du gradient.

In [5]:
class MyReLU(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

## exemple avec MNIST

C'est à ce moment que je choisis pour bifurque sur un autre tutoriel [pytorch et MNIST](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html) qui commence sur un réseau de neurones pour *MNIST*.

In [7]:
dtype = torch.float
device = torch.device("cpu")

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Le gradient

On choisit une entrée aléatoire mais aux mêmes dimensions qu'une image et on calcule la sortie du réseau de neurones.

In [9]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1206, -0.0651,  0.0964, -0.0375, -0.0080, -0.1063,  0.0421, -0.0432,
         -0.0357,  0.0283]], grad_fn=<AddmmBackward>)


Pour calculer le gradient, il faut choisir une erreur... On prend le [MSELoss](https://pytorch.org/docs/stable/nn.html?highlight=mseloss#torch.nn.MSELoss) même si ça n'a rien à voir avec le problème initial mais on s'en fout.

In [10]:
criterion = nn.MSELoss()

On choisit une entrée aléatoire...

In [11]:
target = torch.randn(10)

On calcule une sortie et on redimensionne la cible de sorte qu'elle est la même dimension que la cible.

In [12]:
output = net(input)
target = target.view(1, -1)  # make it the same shape as output
output, target

(tensor([[ 0.1206, -0.0651,  0.0964, -0.0375, -0.0080, -0.1063,  0.0421, -0.0432,
          -0.0357,  0.0283]], grad_fn=<AddmmBackward>),
 tensor([[ 0.3268,  0.6423,  0.4603, -1.7247, -0.6567,  1.0883, -0.8122, -0.6066,
           0.3827,  0.2351]]))

Maintenantn on calcule l'erreur de prédiction...

In [13]:
loss = criterion(output, target)
loss

tensor(0.6635, grad_fn=<MseLossBackward>)

Et on calcule enfin le gradient :

In [14]:
loss.backward()

Je m'attendais à trouver le gradient sur la forme d'un beau vecteur mais comme c'est un réseau de neurones, le gradient est stocké dans chacune des couches sous la forme d'un gradient et encore en deux morceaux...

In [15]:
net.conv1.bias.grad

tensor([-0.0026,  0.0060,  0.0018, -0.0010,  0.0144, -0.0085])

In [16]:
net.conv1.weight.grad

tensor([[[[ 7.7495e-04,  2.0509e-02, -1.5444e-03,  3.2924e-03,  2.8570e-03],
          [ 4.3554e-03, -3.8494e-03, -3.6928e-03, -9.9628e-03, -1.4081e-02],
          [ 1.6582e-02, -3.6654e-03,  7.2560e-03,  4.3759e-03, -9.0065e-03],
          [-4.9111e-03,  3.0753e-03,  1.0345e-02, -2.0349e-02, -5.3538e-03],
          [-2.5743e-02, -2.0375e-03, -8.9529e-04,  1.0883e-02, -3.3448e-03]]],


        [[[-1.0780e-02,  3.5261e-04,  4.1009e-03,  5.8511e-04,  9.1050e-03],
          [ 7.3300e-03,  9.2036e-04,  6.7923e-03,  5.2322e-03,  3.4935e-03],
          [ 6.3614e-03, -9.7264e-04, -1.3278e-03,  1.3456e-02, -1.5942e-02],
          [-8.9549e-03,  1.0247e-02,  5.9081e-03,  5.0274e-03, -3.4880e-03],
          [-1.1709e-02,  8.6279e-03, -5.1750e-03, -7.6851e-03,  1.5142e-02]]],


        [[[-9.6119e-03,  1.9943e-02,  1.0360e-02, -1.0992e-02, -1.4673e-03],
          [ 5.6452e-03, -1.0945e-02, -4.3214e-03, -1.3912e-03,  6.0289e-03],
          [-6.1422e-03, -7.0736e-03,  1.7746e-03,  1.0032e-02,  1.38

## On change de couche

Non, il n'est pas trois heures du matin et personne ne pleure.

In [17]:
class Net0(nn.Module):

    def __init__(self):
        super(Net0, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = MyReLU(self.fc1(x))
        x = MyReLU(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net0 = Net0()
print(net0)

Net0(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Evidemment, ça ne marche jamais du premier coup. On a beaucoup sous-estimé le temps de préparation des cours avec l'informatique. Il est seulement 9h du soir, l'humilité vient seulement après minuit.

In [18]:
try:
    output = net0(input)
except Exception as e:
    print(e)

'MyReLU' object has no attribute 'dim'


J'avoue que j'ai tout lu en diagonal pensant que ma science infinie me permettrait de combler mon ignorance. Honnêtement, il est temps de dîner !

## After the bug...

J'ai craqué, je n'ai pas eu le courage de recoder à 2h du matin trying to figure out what's going wrong. J'ai mélangé tout ce que j'ai pu trouvé et il est quasiment 2h du matin le lendemain.

In [19]:
class MyReLUF(torch.autograd.Function):

    def forward(self, input):
        self.input = input
        res = input.clamp(min=0)
        return res

    def backward(self, grad_output):
        input = self.input
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

    
class MyReLUNN(nn.Module):

    def __init__(self):
        super(MyReLUNN, self).__init__()
        self.fct = MyReLUF()

    def forward(self, input):
        return self.fct(input)

    def extra_repr(self):
        return 'in_features={}'.format(self.in_features)
        

class Net2(nn.Module):

    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fct1 = MyReLUNN()
        self.fct2 = MyReLUNN()

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = self.fct1(self.fc1(x))
        x = self.fct2(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    
net2 = Net2()
output = net2(input)
output

tensor([[ 0.0657,  0.0375, -0.1286,  0.0171, -0.0146,  0.0795,  0.0423,  0.0091,
         -0.0872, -0.1257]], grad_fn=<AddmmBackward>)

In [20]:
target = target.view(1, -1)  # make it the same shape as output
output, target

(tensor([[ 0.0657,  0.0375, -0.1286,  0.0171, -0.0146,  0.0795,  0.0423,  0.0091,
          -0.0872, -0.1257]], grad_fn=<AddmmBackward>),
 tensor([[ 0.3268,  0.6423,  0.4603, -1.7247, -0.6567,  1.0883, -0.8122, -0.6066,
           0.3827,  0.2351]]))

In [21]:
loss = criterion(output, target)
loss

tensor(0.6705, grad_fn=<MseLossBackward>)

In [22]:
loss.backward()

In [23]:
net2.conv1.bias.grad

tensor([ 0.0138, -0.0025, -0.0035, -0.0046,  0.0023, -0.0033])

In [24]:
net2.conv1.weight.grad

tensor([[[[-1.8600e-03, -8.2138e-03,  1.3108e-02,  6.5663e-03,  9.5023e-04],
          [ 5.2586e-03,  1.8312e-02,  3.2803e-03, -1.3340e-03,  8.4217e-03],
          [ 6.3147e-03, -1.0942e-02, -1.1943e-05, -3.3186e-03,  1.0118e-02],
          [ 1.0208e-02, -6.0151e-03, -8.2038e-03, -2.4967e-03, -6.4575e-03],
          [-1.5466e-02, -8.3046e-03,  1.2083e-03, -1.1412e-03, -4.6954e-03]]],


        [[[ 8.1948e-03, -2.9225e-03, -8.8196e-03,  7.8538e-03, -1.8145e-02],
          [-4.1026e-03, -2.1482e-02,  2.5439e-03, -9.5643e-03, -1.5817e-02],
          [ 5.4223e-04, -5.6139e-03,  1.3325e-02,  1.7416e-02, -5.9665e-03],
          [ 1.5479e-04,  1.0664e-02,  5.8910e-03,  7.4956e-03, -3.5761e-03],
          [-1.0230e-02,  5.2740e-03, -4.1645e-03, -1.2707e-03,  1.0064e-02]]],


        [[[ 3.5419e-03,  1.0482e-02, -9.5914e-03, -1.3956e-02,  1.8763e-02],
          [ 5.3130e-03,  3.9722e-03, -2.2886e-03, -1.0493e-02, -1.0075e-03],
          [ 9.7137e-03,  6.4197e-03,  2.3394e-03, -1.1650e-02,  1.46

## Ca converge ?

Il ne reste plus qu'à voir si cela converge.